<a href="https://colab.research.google.com/github/Ron-Rocks/ImageClassification/blob/master/ImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense,Flatten
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16,preprocess_input
import tensorflow as tf

# Setting up The Data
(trainX,trainY),(testX,testY) = cifar10.load_data()

labels = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
trainY = to_categorical(trainY)
testY = to_categorical(testY)
print(trainY.shape)
trainX = trainX/255
testX = testX/255

print(trainX.shape)

Using TensorFlow backend.


170500096/170498071 [==============================] - 6s 0us/step
(50000, 10)
(50000, 32, 32, 3)


# Using The VGG-16 model Fist

In [0]:
model = VGG16(input_shape = (32,32,3,),include_top = False)
model.summary()
# Freezing The model
for layer in model.layers :
  layer.trainable = False
vggOutput =  model.layers[-2]
x = Flatten()(vggOutput.output)
x = Dense(256,activation = "relu",input_shape=(512,))(x)
x = Dense(10,activation = "sigmoid")(x)

model = Model(model.inputs,x)

model.summary()





58892288/58889256 [==============================] - 1s 0us/step






Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
____________________________________________________

In [0]:
callback = tf.keras.callbacks.ProgbarLogger(count_mode="samples")

In [0]:

model.compile(loss = "categorical_crossentropy",optimizer="adam",metrics = ["accuracy"])
model.fit(trainX,trainY,epochs = 20,validation_data=(testX,testY),batch_size=128)
#model.save_weights("weigths1.h5")


In [0]:
%load_ext tensorboard

# **Now Using Resnet**

In [4]:
from keras.applications.resnet import ResNet50

model = ResNet50(input_shape = (32,32,3,),include_top = False)

# Freezing The model
for layer in model.layers :
  layer.trainable = False

x = Flatten()(model.output)
x = Dense(256,activation = "relu",input_shape=(512,))(x)
x = Dense(10,activation = "sigmoid")(x)

model = Model(model.inputs,x)

model.summary()

model.compile(loss = "categorical_crossentropy",optimizer="adam",metrics = ["accuracy"])

model.fit(trainX,trainY,epochs = 20,validation_data=(testX,testY),batch_size=128,callbacks = [callback])
model.save_weights("weightsResnet.h5")

eval = model.evaluate(testX,testY)
print("Evaluation :  loss- ",eval[0],"  Accuracy - ",eval[1])


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________